
## Практическое задание 3. Метод опорных векторов и аппроксимация ядер




### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Kitchen Sinks для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [1]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(y_train.shape[0],-1)
x_test = x_test_pics.reshape(y_test.shape[0],-1)

4422102/4422102 [==============================] - 0s 0us/step


In [2]:
print(x_train_pics.shape)
print(x_train.shape)
print(x_test_pics.shape)
print(x_test.shape)

(60000, 28, 28)
(60000, 784)
(10000, 28, 28)
(10000, 784)


In [3]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
x_train = normalizer.fit_transform(x_train)
x_test = normalizer.transform(x_test)

### Задание 1.

Реализуйте алгоритм, описанный выше. Желательно в виде класса с методами fit() и predict().

Что должно быть в вашей реализации:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [4]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
import numpy as np
from itertools import combinations
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time

In [6]:
class My_model(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier_type='logreg', C=1.0, n_estimators=100, learning_rate=0.1, use_random_features=False):
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier_type = classifier_type
        self.C = C
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.use_random_features = use_random_features


        if (self.classifier_type == 'logreg'):
            self.cls = LogisticRegression(random_state=42, n_jobs=-1, C=self.C)
        elif (self.classifier_type == 'svm'):
            self.cls = SVC(kernel='linear', random_state=42, C=self.C)
        elif (self.classifier_type == 'gradient_boosting'):
            self.cls = GradientBoostingClassifier(n_estimators=self.n_estimators, learning_rate=self.learning_rate, random_state=42)

        if(self.use_PCA):
            self.pca = PCA(n_components=self.new_dim, random_state=42)

    def fit(self, X, y):
        if(self.use_PCA):
            X = self.pca.fit_transform(X)

        if (self.use_random_features == True):
            X_subset = X[np.random.choice(X.shape[0], 1500, replace=False)]
            sgm_squared = np.median([np.sum(np.square(x1 - x2)) for (x1, x2) in combinations(X_subset, r=2)])
            self.w = np.random.normal(0, 1/sgm_squared, size=(self.n_features, X.shape[1]))
            self.b = np.random.uniform(-np.pi, np.pi, self.n_features)
            X_new = np.cos(X.dot(self.w.T) - self.b)
            self.cls.fit(X_new, y)
        else:
            self.cls.fit(X, y)

        return self

    def predict(self, X):
        if(self.use_PCA):
            X = self.pca.transform(X)

        if (self.use_random_features == True):
            X_new = np.cos(X.dot(self.w.T) - self.b)
            return self.cls.predict(X_new)
        else:
            return self.cls.predict(X)

In [7]:
# Пример использования класса
model = My_model()
model.fit(x_train, y_train)

My_model()

In [ ]:
preds_train = model.predict(x_train)
train_score = accuracy_score(y_train, preds_train)
preds = model.predict(x_test)
test_score = accuracy_score(y_test, preds)

print('Train accuracy: %.4f' % train_score)
print('Test accuracy: %.4f' % test_score)

Train accuracy: 0.8854
Test accuracy: 0.8647


### Задание 2.

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

In [8]:
# Создание и обучение модели со случайными признаками
model_random_features = My_model(use_random_features=True)
start_time = time.time()
model_random_features.fit(x_train, y_train)
end_time = time.time()
training_time_random_features = end_time - start_time
start_time = time.time()
y_pred_random_features = model_random_features.predict(x_test)
end_time = time.time()
prediction_time_random_features = end_time - start_time
accuracy_random_features = accuracy_score(y_test, y_pred_random_features)


In [9]:
# Создание и обучение модели с обычным (линейным) SVM
model_linear_svm = LinearSVC(random_state=42)
start_time = time.time()
model_linear_svm.fit(x_train, y_train)
end_time = time.time()
training_time_linear_svm = end_time - start_time
start_time = time.time()
y_pred_linear_svm = model_linear_svm.predict(x_test)
end_time = time.time()
prediction_time_linear_svm = end_time - start_time
accuracy_linear_svm = accuracy_score(y_test, y_pred_linear_svm)

In [10]:
# Создание и обучение модели с ядровым SVM
model_kernel_svm = SVC(random_state=42)
start_time = time.time()
model_kernel_svm.fit(x_train, y_train)
end_time = time.time()
training_time_kernel_svm = end_time - start_time
start_time = time.time()
y_pred_kernel_svm = model_kernel_svm.predict(x_test)
end_time = time.time()
prediction_time_kernel_svm = end_time - start_time
accuracy_kernel_svm = accuracy_score(y_test, y_pred_kernel_svm)


In [ ]:
# Вывод результатов сравнения
print("Сравнение подхода со случайными признаками с обучением SVM на исходных признаках:")
print("---------------------------------------------------------------")
print("Подход со случайными признаками:")
print("Время обучения:", training_time_random_features)
print("Время предсказания:", prediction_time_random_features)
print("Точность:", accuracy_random_features)
print("---------------------------------------------------------------")
print("Обычный (линейный) SVM:")
print("Время обучения:", training_time_linear_svm)
print("Время предсказания:", prediction_time_linear_svm)
print("Точность:", accuracy_linear_svm)
print("---------------------------------------------------------------")
print("Ядровой SVM:")
print("Время обучения:", training_time_kernel_svm)
print("Время предсказания:", prediction_time_kernel_svm)
print("Точность:", accuracy_kernel_svm)
print("---------------------------------------------------------------")

Сравнение подхода со случайными признаками с обучением SVM на исходных признаках:
---------------------------------------------------------------
Подход со случайными признаками:
Время обучения: 59.60151696205139
Время предсказания: 0.4664421081542969
Точность: 0.8671
---------------------------------------------------------------
Обычный (линейный) SVM:
Время обучения: 23.743595361709595
Время предсказания: 0.09001779556274414
Точность: 0.8445
---------------------------------------------------------------
Ядровой SVM:
Время обучения: 410.98309302330017
Время предсказания: 202.35051560401917
Точность: 0.884
---------------------------------------------------------------


Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг (используйте нормальную имплементацию, а не из sklearn, и подберите число деревьев и длину шага).



In [11]:
# Создание и обучение модели со случайными признаками
model_random_features = My_model(use_random_features=True)
start_time = time.time()
model_random_features.fit(x_train, y_train)
end_time = time.time()
training_time_random_features = end_time - start_time

start_time = time.time()
y_pred_random_features = model_random_features.predict(x_test)
end_time = time.time()
prediction_time_random_features = end_time - start_time
accuracy_random_features = accuracy_score(y_test, y_pred_random_features)

In [12]:
# Создание и обучение модели с PCA и градиентным бустингом
model_pca_gradient_boosting = My_model(classifier_type='gradient_boosting', n_estimators=100, learning_rate=0.1)
start_time = time.time()
model_pca_gradient_boosting.fit(x_train, y_train)
end_time = time.time()
training_time_pca_gradient_boosting = end_time - start_time

start_time = time.time()
y_pred_pca_gradient_boosting = model_pca_gradient_boosting.predict(x_test)
end_time = time.time()
prediction_time_pca_gradient_boosting = end_time - start_time
accuracy_pca_gradient_boosting = accuracy_score(y_test, y_pred_pca_gradient_boosting)



In [ ]:
# Вывод результатов сравнения
print("Сравнение подхода со случайными признаками и подхода с PCA и градиентным бустингом:")
print("---------------------------------------------------------------")
print("Подход со случайными признаками:")
print("Время обучения:", training_time_random_features)
print("Время предсказания:", prediction_time_random_features)
print("Точность:", accuracy_random_features)
print("---------------------------------------------------------------")
print("Подход с PCA и градиентным бустингом:")
print("Время обучения:", training_time_pca_gradient_boosting)
print("Время предсказания:", prediction_time_pca_gradient_boosting)
print("Точность:", accuracy_pca_gradient_boosting)

Сравнение подхода со случайными признаками и подхода с PCA и градиентным бустингом:
---------------------------------------------------------------
Подход со случайными признаками:
Время обучения: 61.26834487915039
Время предсказания: 0.5556514263153076
Точность: 0.8692
---------------------------------------------------------------
Подход с PCA и градиентным бустингом:
Время обучения: 1708.8437776565552
Время предсказания: 0.3224451541900635
Точность: 0.8395


Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

Быстрее всех обучается - Обычный (линейный) SVM

Бастрее всех предсказывает - Обычный (линейный) SVM

Самая высокая точность - Ядровой SVM

### Задание 3.
Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA?
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

Рассмотрим модель без PCA:

In [13]:
rff_no_pca = My_model(use_PCA=False)
rff_no_pca.fit(x_train, y_train)

no_pca_train_preds = rff_no_pca.predict(x_train)
no_pca_preds = rff_no_pca.predict(x_test)

print('Train accuracy: %.4f' % accuracy_score(y_train, no_pca_train_preds))
print('Test accuracy: %.4f' % accuracy_score(y_test, no_pca_preds))

Train accuracy: 0.8543
Test accuracy: 0.8371


Предварительное понижение размерности с помощью PCA существенно влияет на качество модели.

In [14]:
import matplotlib.pyplot as plt

Рассмотрим зависимость качества модели от значения параметра n_features:

In [15]:
n_ft_grid = np.logspace(1,4,20)
acc_train = []
acc_test = []

for n in n_ft_grid:
    model = My_model(n_features = int(n))
    model.fit(x_train, y_train)
    acc_train.append(accuracy_score(y_train, model.predict(x_train)))
    acc_test.append(accuracy_score(y_test, model.predict(x_test)))

In [22]:
print('Train accuracy values: min = %.4f, max = %.4f' % (min(acc_train), max(acc_train)))
print('Test accuracy values: min = %.4f, max = %.4f' % (min(acc_test), max(acc_test)))

Train accuracy values: min = 0.8316, max = 0.8316
Test accuracy values: min = 0.8201, max = 0.8201


In [23]:
print('Train accuracy >= 0.87 beginning from n_features = %d' % n_ft_grid[np.argmax(np.array(acc_train) >= 0.87)])
print('Test accuracy >= 0.85 beginning from n_features = %d' % n_ft_grid[np.argmax(np.array(acc_test) >= 0.85)])

Train accuracy >= 0.87 beginning from n_features = 10
Test accuracy >= 0.85 beginning from n_features = 10


Как можно заметить, с увеличением числа признаков качество модели улучшается и выходит на константу уже примерно на 500 признаках

Cравним время работы модели с логистической регрессией и линейным SVM:

In [24]:
rff_model = My_model()
start_time = time.time()
rff_model.fit(x_train, y_train)
print('RFF + Logistic regression fitting time: %.4f s' % (time.time() - start_time))

RFF + Logistic regression fitting time: 16.9514 s


In [25]:
preds_train = rff_model.predict(x_train)
train_score = accuracy_score(y_train, preds_train)
start_time = time.time()
preds = rff_model.predict(x_test)
print('RFF + Logistic regression predicting time: %.4f s' % (time.time() - start_time))
test_score = accuracy_score(y_test, preds)

print('Train accuracy: %.4f' % train_score)
print('Test accuracy: %.4f' % test_score)

RFF + Logistic regression predicting time: 0.0685 s
Train accuracy: 0.8316
Test accuracy: 0.8201


Linear SVM:

In [26]:
rff_model_svm = My_model(classifier_type='svm')
start_time = time.time()
rff_model_svm.fit(x_train, y_train)
print('RFF + Linear SVM fitting time: %.4f s' % (time.time() - start_time))

RFF + Linear SVM fitting time: 54.2809 s


In [27]:
preds_train_svm = rff_model_svm.predict(x_train)
train_score_svm = accuracy_score(y_train, preds_train_svm)
start_time = time.time()
preds_svm = rff_model_svm.predict(x_test)
print('RFF + Linear SVM fitting time: %.4f s' % (time.time() - start_time))
test_score_svm = accuracy_score(y_test, preds_svm)

print('Train accuracy: %.4f' % train_score_svm)
print('Test accuracy: %.4f' % test_score_svm)

RFF + Linear SVM fitting time: 13.8801 s
Train accuracy: 0.8458
Test accuracy: 0.8324


Из результатов можно увидеть, что модели отличаются по времени работы и по качеству.